### LSE Data Analytics Online Career Accelerator

# DA201: Data Analytics using Python

## Obtain current and historical weather data  (demonstration)

**Important**

*Please take note that you will work with the OpenWeather API. Keep in mind that the API is based on live and current weather conditions and specified locations.*

Previously, you learned how to set up your OpenWeather account, create a Environment variable for your API key, import the key into your Jupyter Notebook, and generate a simple request. Let's investigate how to obtain, format, and compare data for the weather in your location today, and in the year 2000. Once you are comfortable, you can then define and compare multiple locations.

# 1. Prepare your workstation

In [2]:
# Import libraries.
import os
import requests
import pandas as pd
import json
from dotenv import load_dotenv

load_dotenv() 

# Locate and read the key from your .env file.
API_key = os.getenv('OpenWeather_API_key')


# 2. Find your latitude and longitude coordinates

In [5]:
# Create an API call.
coordinates = "http://api.openweathermap.org/geo/1.0/zip?zip=G71,GB" "&appid=" + API_key

In [6]:
# Define a response variable.
response = requests.get(coordinates)

# Print the response.
print("My coordinates:" , response.json())

My coordinates: {'zip': 'G71', 'name': 'South Lanarkshire', 'lat': 55.8112, 'lon': -4.076, 'country': 'GB'}


# 3. Retrieve your current weather

In [9]:
# Create an API call for the current weather in your location and set the parameter to Celsius.
my_weather = "https://api.openweathermap.org/data/3.0/onecall?lat=51.3824&lon=1.3367&exclude=minutely,hourly,daily&units=metric" "&appid=" + API_key

# Define a response variable.
response = requests.get(my_weather)

# Print the response.
print("My weather:" , response.json())

My weather: {'lat': 51.3824, 'lon': 1.3367, 'timezone': 'Europe/London', 'timezone_offset': 3600, 'current': {'dt': 1690709507, 'sunrise': 1690690512, 'sunset': 1690746409, 'temp': 20.39, 'feels_like': 20, 'pressure': 1011, 'humidity': 58, 'dew_point': 11.86, 'uvi': 4.51, 'clouds': 31, 'visibility': 10000, 'wind_speed': 5.36, 'wind_deg': 221, 'wind_gust': 7.6, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}]}}


In [11]:
# Select and print specified data.
data = response.json()['current']

humidity = data['humidity']
pressure = data['pressure']
wind = data['wind_speed']
description = data['weather'][0]['description']
temp = data['temp']

#Create a bolded title for your specified data output.
print('\033[1mSelected data\033[0m')

print('Temperature:', temp, '°C')
print('Wind:', wind)
print('Pressure:', pressure)
print('Humidity:', humidity)
print('Description:', description)

Selected data
Temperature: 20.29 °C
Wind: 5.36
Pressure: 1011
Humidity: 58
Description: scattered clouds


# 4. Create a Data Frame

In [12]:
# Prepare the Data Frame by creating a list of dictionaries containing the weather data.
weather_data = [{
    'timestamp': pd.Timestamp(data['dt'], unit='s', tz='Europe/London'),
    'temperature': data['temp'],
    'humidity': data['humidity'],
    'pressure': data['pressure'],
    'wind_speed': data['wind_speed'],
    'description': data['weather'][0]['description']
}]

# Create a data frame from the list of dictionaries.
df1 = pd.DataFrame(weather_data)
# Print the data frame.
print(df1)

                  timestamp  temperature  humidity  pressure  wind_speed  \
0 2023-07-30 10:33:34+01:00        20.29        58      1011        5.36   

        description  
0  scattered clouds  


# 5. Retrieve historical weather data for your location

In [16]:
# Create an API call for the current weather in your location and set the parameter to Celsius.
past_weather = "https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=55.8112&lon=-4.076&dt=957471056&units=metric" "&appid=" + API_key

# Define a response variable.
response = requests.get(past_weather)

# Print the response.
print("My weather in 2000:" , response.json())

My weather in 2000: {'lat': 55.8112, 'lon': -4.076, 'timezone': 'Europe/London', 'timezone_offset': 3600, 'data': [{'dt': 957471056, 'sunrise': 957414317, 'sunset': 957470441, 'temp': 10.98, 'feels_like': 9.91, 'pressure': 1025, 'humidity': 68, 'dew_point': 5.31, 'clouds': 2, 'wind_speed': 2.66, 'wind_deg': 75, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}]}]}


In [17]:
# Select and print specified data. 
data = response.json()['data'][0]

humidity = data['humidity']
pressure = data['pressure']
wind = data['wind_speed']
description = data['weather'][0]['description']
temp = data['temp']

print('Temperature:', temp, '°C')
print('Wind:', wind)
print('Pressure:', pressure)
print('Humidity:', humidity)
print('Description:', description)

Temperature: 10.98 °C
Wind: 2.66
Pressure: 1025
Humidity: 68
Description: clear sky


In [18]:
# Prepare a Data Frame by creating a list of dictionaries containing the weather data.
data = json.loads(response.content)['data']

weather_data = []

for datum in data:
    weather_data.append({
        'timestamp': pd.Timestamp(datum['dt'], unit='s', tz='Europe/London'),
        'temperature': datum['temp'],
        'humidity': datum['humidity'],
        'pressure': datum['pressure'],
        'wind_speed': datum['wind_speed'],
        'description': datum['weather'][0]['description']
    })

df2 = pd.DataFrame(weather_data)

print(df2)

                  timestamp  temperature  humidity  pressure  wind_speed  \
0 2000-05-04 21:10:56+01:00        10.98        68      1025        2.66   

  description  
0   clear sky  


# 6. Joining the two DataFrames

In [19]:
# Concatenate the DataFrames.
df_concatenated = pd.concat([df1, df2])

# View the output.
print(df_concatenated)

                  timestamp  temperature  humidity  pressure  wind_speed  \
0 2023-07-30 10:33:34+01:00        20.29        58      1011        5.36   
0 2000-05-04 21:10:56+01:00        10.98        68      1025        2.66   

        description  
0  scattered clouds  
0         clear sky  


# 7. Retrieve coordinates using a location name

In [31]:
holiday_data = "http://api.openweathermap.org/geo/1.0/direct?q=Abu Dhabi&limit=1" "&appid=" + API_key

resp3 = requests.get(holiday_data)
print("Holiday coordinates:", resp3.json())

Holiday coordinates: [{'name': 'Abu Dhabi', 'local_names': {'yo': 'Abu Dhabi', 'sl': 'Abu Dabi', 'am': 'አቡ ዳቢ', 'ht': 'Abou Dabi', 'de': 'Abu Dhabi', 'sk': 'Abú Zabí', 'sc': 'Abu Dhabi', 'id': 'Abu Dhabi', 'lt': 'Abu Dabis', 'ka': 'აბუ-ზაბი', 'fo': 'Abu Dhabi', 'tt': 'Әбү-Дәби', 'kn': 'ಅಬು ಧಾಬಿ', 'sr': 'Абу Даби', 'gd': 'Abu Dhabi', 'ks': 'ابوظہبی', 'nl': 'Abu Dhabi', 'th': 'อาบูดาบี', 'ku': 'Abû Zebî', 'ps': 'ابوظبی', 'is': 'Abú Dabí', 'uk': 'Абу-Дабі', 'tl': 'Abu Dhabi', 'os': 'Абу-Даби', 'bo': 'ཨ་པོའུ་དྷ་པེ།', 'vi': 'Abu Dhabi', 'sv': 'Abu Dhabi', 'ml': 'അബുദാബി', 'br': 'Abu Dhabi', 'so': 'Abu Dabi', 'fi': 'Abu Dhabi', 'en': 'Abu Dhabi', 'ie': 'Abu Dhabi', 'fy': 'Abû Daby', 'he': 'אבו דאבי', 'eu': 'Abu Dhabi', 'ur': 'ابوظبی', 'ce': 'Абу-Даби', 'el': 'Αμπού Ντάμπι', 'et': 'Abu Dhabi', 'hi': 'अबू धाबी', 'ga': 'Abú Daibí', 'af': 'Aboe Dhabi', 'bs': 'Abu Dabi', 'sq': 'Abu Dabi', 'kk': 'Абу-Даби', 'mr': 'अबु धाबी', 'ko': '아부다비', 'hy': 'Աբու Դաբի', 'oc': 'Abu Dhabi', 'sn': 'Abu Dhabi', 'u

# 8. Using the keys() method

In [24]:
# Create an API call using the coordinates retrived using a location name.
holiday_data = "https://api.openweathermap.org/data/3.0/onecall?lat=40.7029134&lon=-73.8006429&exclude=minutely,hourly,daily&units=metric" "&appid=" + API_key


# Define a response variable.
newresp = requests.get(holiday_data)

# Print the response keys.
print(newresp.json().keys())

# Print the response.
print("Today's weather in my holiday destination:" , newresp.json())


dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'current'])
Today's weather in my holiday destination: {'lat': 40.7029, 'lon': -73.8006, 'timezone': 'America/New_York', 'timezone_offset': -14400, 'current': {'dt': 1690710377, 'sunrise': 1690710579, 'sunset': 1690762406, 'temp': 19.13, 'feels_like': 19.03, 'pressure': 1011, 'humidity': 74, 'dew_point': 14.39, 'uvi': 0, 'clouds': 40, 'visibility': 10000, 'wind_speed': 8.23, 'wind_deg': 340, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}]}}


In [25]:
# Access the weather data using the 'current' key.
current = newresp.json()['current']

# Extract the relevant weather information.
humidity = current['humidity']
pressure = current['pressure']
wind = current['wind_speed']
description = current['weather'][0]['description']
temp = current['temp']
feels_like = current['feels_like']
uvi = current['uvi']
clouds = current['clouds']
visibility = current['visibility']
dew_point = current['dew_point']
wind_deg = current['wind_deg']

#Format a heading.
print('\033[1mSelected data\033[0m')

# Print the weather information.
print('Temperature:', round(temp), '°C')
print('Feels like:', round(feels_like), '°C')
print('Wind speed:', round(wind), 'm/s')
print('Wind degree:', wind_deg, '°')
print('Pressure:', pressure, 'hPa')
print('Humidity:', humidity, '%')
print('Dew point:', round(dew_point), '°C')
print('UV index:', uvi)
print('Cloud cover:', clouds, '%')
print('Visibility:', visibility, 'm')
print('Description:', description)

Selected data
Temperature: 19 °C
Feels like: 19 °C
Wind speed: 8 m/s
Wind degree: 340 °
Pressure: 1011 hPa
Humidity: 74 %
Dew point: 14 °C
UV index: 0
Cloud cover: 40 %
Visibility: 10000 m
Description: scattered clouds


# 9. Making multiple API calls simultaneously

In [32]:
import os
import requests
import pandas as pd
import pytz
from datetime import datetime

# Set the API endpoint URL and API key.
endpoint_url = "https://api.openweathermap.org/data/3.0/onecall"
#Access your API key.
API_key = os.getenv('OpenWeather_API_key')

# Define a list of coordinates for different locations
locations = [
    {'name': 'London', 'lat': 51.5074, 'lon': -0.1278},
    {'name': 'Paris', 'lat': 48.8566, 'lon': 2.3522},
    {'name': 'New York', 'lat': 40.7128, 'lon': -74.0060},
    {'name': 'Tokyo', 'lat': 35.6895, 'lon': 139.6917},
    {'name': 'Kampala', 'lat': 0.31771, 'lon': 32.5813539},
    {'name': 'Glasgow', 'lat': 55.8609825, 'lon': -4.2488787},
    {'name': 'Abu Dhabi', 'lat': 24.4538352, 'lon': 54.3774014}
]

# Create an empty list to store the weather data for each location.
weather_data = []

# Loop through each location and get its weather data.
for loc in locations:
    # Set the API parameters.
    params = {
        "lat": loc['lat'],
        "lon": loc['lon'],
        "units": "metric",
        "exclude": "minutely,hourly,daily",
        "appid": API_key
    }

    # Send a GET request to the API endpoint and store the response.
    response = requests.get(endpoint_url, params=params)

    # Access the current weather data.
    current = response.json()['current']

    # Extract the relevant weather information.
    humidity = current['humidity']
    pressure = current['pressure']
    wind = current['wind_speed']
    description = current['weather'][0]['description']
    temp = int(round(current['temp']))
    sunrise = datetime.fromtimestamp(current['sunrise'], tz=pytz.utc).astimezone(pytz.timezone('Europe/London'))
    sunset = datetime.fromtimestamp(current['sunset'], tz=pytz.utc).astimezone(pytz.timezone('Europe/London'))

    # Add the weather data to the list.
    weather_data.append({
        'Location': loc['name'],
        'Temperature (°C)': temp,
        'Wind Speed (m/s)': wind,
        'Pressure (hPa)': pressure,
        'Humidity (%)': humidity,
        'Description': description,
        'Sunrise': sunrise,
        'Sunset': sunset
    })

# Print the weather data in a tabular format.
print("Location\tTemperature (°C)\tWind Speed (m/s)\tPressure (hPa)\tHumidity (%)\tDescription\tSunrise\t\t\t\tSunset")
for data in weather_data:
    print(f"{data['Location']}\t\t{data['Temperature (°C)']}\t\t{data['Wind Speed (m/s)']}\t\t{data['Pressure (hPa)']}\t\t{data['Humidity (%)']}\t\t{data['Description']}\t{data['Sunrise'].strftime('%Y-%m-%d %H:%M:%S')}\t{data['Sunset'].strftime('%Y-%m-%d %H:%M:%S')}")
    

Location	Temperature (°C)	Wind Speed (m/s)	Pressure (hPa)	Humidity (%)	Description	Sunrise				Sunset
London		20		7.72		1010		59		broken clouds	2023-07-30 05:20:34	2023-07-30 20:53:10
Paris		20		5.66		1017		64		overcast clouds	2023-07-30 05:20:36	2023-07-30 20:33:17
New York		19		7.15		1012		70		scattered clouds	2023-07-30 10:50:27	2023-07-31 01:14:17
Tokyo		31		9.77		1012		76		scattered clouds	2023-07-29 20:46:55	2023-07-30 10:48:16
Kampala		28		3.56		1014		44		overcast clouds	2023-07-30 04:52:06	2023-07-30 16:59:58
Glasgow		14		7.72		1000		86		light rain	2023-07-30 05:17:12	2023-07-30 21:29:29
Abu Dhabi		40		2.06		998		37		clear sky	2023-07-30 02:49:41	2023-07-30 16:08:00


# 10. Create a formatted DataFrame and export it as CSV

In [34]:
# Create a data frame from the list of dictionaries.
df = pd.DataFrame(weather_data)

styled_df = df.style \
    .set_caption("Example DataFrame") \
    .set_properties(**{'text-align': 'center'}) \
    .set_table_styles([{'selector': 'th', 'props': [('text-align', 'center')]}]) \
    .background_gradient(cmap='viridis', low=0, high=1)

# display the styled DataFrame
display(styled_df)

# export dataframe to csv
df.to_csv('mydata.csv', index=False)

,Location,Temperature (°C),Wind Speed (m/s),Pressure (hPa),Humidity (%),Description,Sunrise,Sunset
0,London,20,7.720000,1010,59,broken clouds,2023-07-30 05:20:34+01:00,2023-07-30 20:53:10+01:00
1,Paris,20,5.660000,1017,64,overcast clouds,2023-07-30 05:20:36+01:00,2023-07-30 20:33:17+01:00
2,New York,19,7.150000,1012,70,scattered clouds,2023-07-30 10:50:27+01:00,2023-07-31 01:14:17+01:00
3,Tokyo,31,9.770000,1012,76,scattered clouds,2023-07-29 20:46:55+01:00,2023-07-30 10:48:16+01:00
4,Kampala,28,3.560000,1014,44,overcast clouds,2023-07-30 04:52:06+01:00,2023-07-30 16:59:58+01:00
5,Glasgow,14,7.720000,1000,86,light rain,2023-07-30 05:17:12+01:00,2023-07-30 21:29:29+01:00
6,Abu Dhabi,40,2.060000,998,37,clear sky,2023-07-30 02:49:41+01:00,2023-07-30 16:08:00+01:00
